**GAN**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!git clone https://github.com/AvonYangXX1/AMPLify-Feedback.git
import tensorflow as tf
import numpy as np
import os

fatal: destination path 'AMPLify-Feedback' already exists and is not an empty directory.


In [6]:
# Generator
def create_autoencoder(seq_length, depth, latent_dim):
    inputs0 = tf.keras.layers.Input(shape=(seq_length,depth), name="Input0")
    # x = tf.keras.layers.LSTM(512, return_sequences=True, name="Encoder_0")(inputs0)
    # x = tf.keras.layers.LSTM(256, return_sequences=True, name="Encoder_1")(x)
    # x = tf.keras.layers.LSTM(128, return_sequences=True, name="Encoder_2")(x)
    x = tf.keras.layers.Conv1D(32, 5, name="Encoder_0")(inputs0)
    x = tf.keras.layers.Conv1D(32, 5, name="Encoder_1")(x)
    x = tf.keras.layers.Conv1D(32, 5, name="Encoder_2")(x)
    x = tf.keras.layers.GRU(latent_dim, return_sequences=False, name="Encoder_3")(x)
    x = tf.keras.layers.RepeatVector(seq_length, name="RepeatVector")(x)
    x = tf.keras.layers.GRU(128, return_sequences=True, name="Decoder_0")(x)
    x = tf.keras.layers.GRU(128, return_sequences=True, name="Decoder_1")(x)
    x = tf.keras.layers.GRU(128, return_sequences=True, name="Decoder_2")(x)
    x = tf.keras.layers.Dense(depth, activation="softmax", name="Output")(x)
    model = tf.keras.models.Model(inputs=inputs0, outputs=x)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [3]:
aa_vocal = np.load("AMPLify-Feedback/model_weights/SeqTV_vocal.npy")
pep_decoder = tf.keras.layers.StringLookup(vocabulary=aa_vocal[1:], invert=True, oov_token='')

In [4]:
def onehot2seq(onehot):
    seq = tf.math.argmax(onehot, axis=2)
    seq = pep_decoder(seq).numpy().astype('str')
    seq = ["".join(chars) for chars in seq]
    return seq

In [ ]:
latent_dim = 2
seq_length = 190
depth = 43
np.random.seed(8701)
demo_noise = (np.random.rand(1, latent_dim)-0.5)*2
autoencoder = create_autoencoder(seq_length, depth, latent_dim)
path = "AMPLify-Feedback/processed_data/gan_train_data"
files = os.listdir(path)
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for file in files:
        seq = np.load(f"{path}/{file}", allow_pickle=True)
        seq = tf.one_hot(seq.squeeze(), depth=43)
        autoencoder.fit(seq, seq, epochs=1, batch_size=22, verbose=1, validation_split=0.2)
        del seq
        autoencoder.save("drive/MyDrive/MIT687/PeptideAutoencoder.keras")
        decoder = tf.keras.models.Model(inputs=autoencoder.layers[5].input, outputs=autoencoder.layers[-1].output)
        demo_seq = decoder(demo_noise)
        del decoder
        demo_seq = onehot2seq(demo_seq)
        print(demo_seq[0])

Epoch 1/10
810/810 [==============================] - 44s 43ms/step - loss: 2.0093 - accuracy: 0.4039 - val_loss: 1.8677 - val_accuracy: 0.4170
MMLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLKKK
810/810 [==============================] - 33s 41ms/step - loss: 1.8778 - accuracy: 0.4153 - val_loss: 1.8823 - val_accuracy: 0.4133
MMLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAKKKKKKK
810/810 [==============================] - 31s 38ms/step - loss: 1.8729 - accuracy: 0.4152 - val_loss: 1.9425 - val_accuracy: 0.4194
MAKLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAKKK
810/810 [==============================] - 30s 37ms/step - loss: 1.8712 - accuracy: 0.4149 - val_loss: 1.8577 - val_accuracy: 0.4182
MAKKLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLKKKKKKKKK
810/810 [==============================] - 32s 39ms/step - loss: 1.8596 

In [ ]:
# After GAN is trained
def generate_sequences(generator, latent_dim, num_sequences):
    noise = np.random.normal(0, 1, (num_sequences, latent_dim))
    generated_sequences = generator.predict(noise, verbose=0)
    return generated_sequences

In [ ]:
num_sequences=10
generated_seqs = generate_sequences(generator, latent_dim, num_sequences=num_sequences)

In [ ]:
generator.save("AMPLify-Feedback/model_weights/PeptideGenerator.keras")

In [ ]:
# np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_10",generated_seqs)

In [ ]:
#Convert to One_hot
# generated_seqs_one_hot = tf.one_hot(generated_seqs.squeeze(), depth=43)
# np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_one_hot_10",generated_seqs_one_hot)